In [1]:
def Shinethrough(df, k):
    df=df.groupby('classifier').head(k)   
    df = df.loc[(df['true_label'] == 1) & (df['pred_label'] == 1)] #To keep only TP entries and remove rest
    df.drop(columns = ['pred_label','true_label'])
    #calculate total number of total unique TP found by all classifiers
    total_unique_TP = len(pd.unique(df['Instance ID']))
    
    ###############################################################################################################
    #No of unique classifiers
    un_class = df['classifier'].unique()
    un_class = sorted(un_class)
    data =[]
    column_names = ['x_axis', 'y_axis']

    #create pairs of classifiers to form a dataframe
    for i in range(len(un_class)):
        for j in range(len(un_class)):
            if i!=j:
                new_ele = (un_class[i],un_class[j])
                data.append(new_ele)
                
    #Dataframe df_all contains count of total unique True positives
    df_all = pd.DataFrame.from_records(
        data, columns=column_names)
    df_all['count'] = total_unique_TP
    
    ###############################################################################################################
    #df_unique_ID contains all the unique True positive reviews(ID)
    df_uni = pd.DataFrame.from_records(data, columns=column_names)
    names=df['classifier'].unique().tolist()
    df_unique_ID = pd.DataFrame()
    for name1 in names:
        df_1 = df.loc[df.classifier==name1] 
        for name2 in names:
            if name1 != name2:
                df_2 = df.loc[df.classifier==name2]
                df_1 = df_1[~df_1['Instance ID'].isin(df_2['Instance ID'])]

        df_unique_ID=df_unique_ID.append(df_1)
    df_unique_ID = df_unique_ID.drop(columns = ['pred_label','true_label'])  

    #df_uni contains count of unique ID's of classifier on y axis
    for name in names:
        df_1 = df_unique_ID.loc[df_unique_ID.classifier==name] 
        df_uni.loc[df_uni['y_axis'] == name, 'count'] = len(df_1.index)
    
    ###############################################################################################################
    #create df_twin containing total unique TP found by classifier on x-axis and y-axis
    df_twin = pd.DataFrame.from_records(data, columns=column_names)
    names=df['classifier'].unique().tolist()
    for name1 in names:
        df_1 = df.loc[df.classifier==name1] 
        for name2 in names:
            if name1 != name2:
                df_2 = df.loc[df.classifier==name2]
                df_two = df_1.append(df_2)
                for name in names:
                    if name !=name1 and name !=name2:
                        df_3 = df.loc[df.classifier==name] 
                        df_two = df_two[~df_two['Instance ID'].isin(df_3['Instance ID'])]
                        n = len(pd.unique(df_two['Instance ID']))
                        df_twin.loc[(df_twin['x_axis'] == name1) & (df_twin['y_axis'] == name2), 'common'] = n
    df_twin = df_twin.rename(columns={'common': 'count'})
    
    ###############################################################################################################
    #shineThrough scores calculations
    def trim_fraction(text):
        text = str(text)
        sep='.'
        stripped = text.split(sep, 1)[0]
        return stripped 

    df_shine_scores =df_uni.drop(columns = ['count'])
    df_shine_scores['shine_score1'] = df_uni['count'].apply(trim_fraction)
    df_shine_scores['shine_score2'] = total_unique_TP
    df_shine_scores['shine_score2'] = df_shine_scores['shine_score2'].apply(trim_fraction)
    df_shine_scores['shine_score3'] = df_twin['count'].apply(trim_fraction)
    df_shine_scores['shine_score4'] = total_unique_TP
    df_shine_scores['shine_score4'] = df_shine_scores['shine_score4'].apply(trim_fraction)

    df_shine_scores['shine_score'] = df_shine_scores['shine_score1'].astype(str)+'/'+df_shine_scores['shine_score2'].astype(str) +' | '+df_shine_scores['shine_score3'].astype(str)+'/'+df_shine_scores['shine_score4'].astype(str)
    df_shine_scores['shine_score'] = df_shine_scores['shine_score'].fillna(0)
    print("ShineThrough Scores(Count of Exclusive True Positives)")
    df_shine_scores = df_shine_scores.pivot(index="y_axis", columns="x_axis", values="shine_score").fillna(value = '-')
    df_shine_scores= df_shine_scores.style.set_properties(**{'text-align': 'center'})
    df_shine_scores = df_shine_scores.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df_shine_scores)
    print("\n")
    print("\n")
    
    ###############################################################################################################
    #shineThrough metric calculations
    def trim_fraction_(text):
        text = str(text)
        sep='.00'
        stripped = text.split(sep, 1)[0]
        return stripped  
    
    df_shine_metric = df_uni.drop(columns = ['count'])
    df_shine_metric['shine_score1'] = df_uni['count']/total_unique_TP
    df_shine_metric['shine_score2'] = df_twin['count']/total_unique_TP
    df_shine_metric = df_shine_metric.round(2)
    df_shine_metric['shine_score1'] = df_shine_metric['shine_score1'].apply(trim_fraction_)
    df_shine_metric['shine_score2'] = df_shine_metric['shine_score2'].apply(trim_fraction_)
    df_shine_metric['shine_score'] = df_shine_metric['shine_score1'].astype(str)+' | '+df_shine_metric['shine_score2'].astype(str)
    print("ShineThrough Metric")
    df_shine_metric= df_shine_metric.pivot(index="y_axis", columns="x_axis", values="shine_score").fillna(value = '-')
    df_shine_metric= df_shine_metric.style.set_properties(**{'text-align': 'center'})
    df_shine_metric = df_shine_metric.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df_shine_metric) 
    print("\n")
    print("\n")
    
    ###############################################################################################################
    #Plotting bubble chart
    #Creating blank dataframe for plotting chart in order
    data1=[]
    for i in range(len(un_class)):
        for j in range(len(un_class)-1, -1, -1):
            new_ele = (un_class[i],un_class[j])
            data1.append(new_ele)
                #data.append(new_ele)

    df_blank = pd.DataFrame.from_records(
        data1, columns=column_names)
    df_blank['count'] =0

    df_twin = df_twin.sort_values(by =['count'],ascending=False)
    df_twin = df_blank.append(df_twin)
    #
    df_twin['size']=df_twin['count']*df_twin['count']
    df_twin['Color Significance']='Exclusive TP found by classifier on x & y axis'
    df_uni['size']=df_uni['count']*df_uni['count']
    df_uni['Color Significance']='Exclusive TP found by classifier on y axis'
    df_final = df_twin.append(df_uni)

    fig=px.scatter(df_final, x="x_axis", y="y_axis",
               color_discrete_sequence=px.colors.qualitative.Alphabet,
               color = 'Color Significance',
               size='size',opacity=0.6,size_max=70,
               color_discrete_map={"Exclusive TP found by classifier on x & y axis": 'orange',"Exclusive TP found by classifier on y axis":'yellow'},hover_data=['Color Significance'],
              )

    fig.update_layout(
        width=1000,
        height=600
    )
    fig.update_layout(
        xaxis={'side': 'top'}, 
        yaxis={'side': 'left'}  
    )
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="top",
        y=0,
        xanchor="left",
        x=0.1
    ))
    fig.update_layout(title_text='MARS ShineThrough Chart', title_x=0.05)
    fig.show()

In [2]:
def Occlusion(df, k):
    df=df.groupby('classifier').head(k)
    df_tp = df.loc[(df['true_label'] == 1) & (df['pred_label'] == 1)]
    df_fn = df.loc[(df['true_label'] == 1) & (df['pred_label'] == 0)]

    #calculate total number of total unique TP found by all classifiers
    total_unique_TP = len(pd.unique(df_tp['Instance ID']))

    un_class = df['classifier'].unique()
    un_class = sorted(un_class)
    data =[]
    column_names = ['x_axis', 'y_axis']   
    #create pairs of classifiers
    for i in range(len(un_class)):
        for j in range(len(un_class)):
            if i!=j:
                new_ele = (un_class[i],un_class[j])
                data.append(new_ele)
                
    #df_all contains total number of unique True positives
    df_all = pd.DataFrame.from_records(data, columns=column_names)
    df_all['count'] = total_unique_TP
    
    df_fn_twin = pd.DataFrame.from_records(data, columns=column_names)

    df_fn_unique1 = pd.DataFrame.from_records(data, columns=column_names)



    names=df['classifier'].unique().tolist()
    df_fn_unique = pd.DataFrame()
    for name1 in names:
        df_1 = df_fn.loc[df_fn.classifier==name1] 
        df_1 = df_1[df_1['Instance ID'].isin(df_tp['Instance ID'])]
        df_fn_unique=df_fn_unique.append(df_1)
        
    ###############################################################################################################
    #Unique FN for that classifier relative to the TP of all the classifiers
    df_fn_unique = df_fn_unique.drop(columns = ['pred_label','true_label'])  

    for name in names:
        df_1 = df_fn_unique.loc[df_fn_unique.classifier==name] 
        df_fn_unique1.loc[df_fn_unique1['y_axis'] == name, 'count'] = len(df_1.index)

    for name1 in names:
        df_1 = df_fn.loc[df_fn.classifier==name1] 
        df_1_tp = df_tp.loc[df_tp.classifier==name1]
        for name2 in names:
            if name1 != name2:           
                df_2 = df_fn.loc[df_fn.classifier==name2]
                df_2_tp = df_tp.loc[df_tp.classifier==name2]
                df_1 = df_1[~df_1['Instance ID'].isin(df_2_tp['Instance ID'])]
                df_2 = df_2[~df_2['Instance ID'].isin(df_1_tp['Instance ID'])]

                df_2 = df_2.append(df_1)
                df_2 = df_2.drop_duplicates(subset='Instance ID', keep="last")
                df_temp = pd.DataFrame()
                for name3 in names:
                    if name3 != name1 and name3 != name2:
                        df_3 = df_tp.loc[df_tp.classifier==name3]
                        df_temp = df_temp.append(df_2[df_2['Instance ID'].isin(df_3['Instance ID'])])
                        df_temp = df_temp.drop_duplicates(subset='Instance ID', keep="last")
                df_fn_twin.loc[(df_fn_twin['y_axis'] == name1) & (df_fn_twin['x_axis'] == name2), 'common'] = len(df_temp.index)
                                                                                                                                          
    df_fn_twin = df_fn_twin.rename(columns={'common': 'count'})
    
    ###############################################################################################################
    #created blank dataframe to arrange classifiers in order for plot
    data1=[]
   
    for i in range(len(un_class)):
        for j in range(len(un_class)-1, -1, -1):
            new_ele = (un_class[i],un_class[j])
            data1.append(new_ele)

    df_blank = pd.DataFrame.from_records(data1, columns=column_names)
    df_blank['count'] =0
    
    ###############################################################################################################
    #Occlusion scores calculations
    def trim_fraction(text):
        text = str(text)
        sep='.'
        stripped = text.split(sep, 1)[0]
        return stripped
    df_occ_scores =df_fn_unique1.drop(columns = ['count'])
    df_occ_scores['occ_score1'] = df_fn_unique1['count']
    df_occ_scores['occ_score2'] = total_unique_TP
    df_occ_scores['occ_score3'] = df_fn_twin['count']
    df_occ_scores['occ_score4'] = total_unique_TP

    df_occ_scores['occ_score1'] = df_occ_scores['occ_score1'].apply(trim_fraction)
    df_occ_scores['occ_score2'] = df_occ_scores['occ_score2'].apply(trim_fraction)
    df_occ_scores['occ_score3'] = df_occ_scores['occ_score3'].apply(trim_fraction)
    df_occ_scores['occ_score4'] = df_occ_scores['occ_score4'].apply(trim_fraction)

    df_occ_scores['occ_score'] = df_occ_scores['occ_score1'].astype(str)+'/'+df_occ_scores['occ_score2'].astype(str) +' | '+df_occ_scores['occ_score3'].astype(str)+'/'+df_occ_scores['occ_score4'].astype(str)
    df_occ_scores['occ_score'] = df_occ_scores['occ_score'].fillna(0)
    print("Occlusion Scores(Count of Exclusive False Negatives)")
    df_occ_scores = df_occ_scores.pivot(index="y_axis", columns="x_axis", values="occ_score").fillna(value = '-')
    df_occ_scores= df_occ_scores.style.set_properties(**{'text-align': 'center'})
    df_occ_scores = df_occ_scores.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df_occ_scores)
    print("\n")
    print("\n")
    ###############################################################################################################
    #Occlusion metric calculations
    def trim_fraction_(text):
        text = str(text)
        sep='.00'
        stripped = text.split(sep, 1)[0]
        return stripped
    df_occ_metric =df_fn_unique1.drop(columns = ['count'])
    df_occ_metric['occ_score1'] = df_fn_unique1['count']/total_unique_TP
    df_occ_metric['occ_score2'] = df_fn_twin['count']/total_unique_TP
    df_occ_metric = df_occ_metric.round(2)

    df_occ_metric['occ_score1'] = df_occ_metric['occ_score1'].apply(trim_fraction_)
    df_occ_metric['occ_score2'] = df_occ_metric['occ_score2'].apply(trim_fraction_)
    df_occ_metric['occ_score'] = df_occ_metric['occ_score1'].astype(str)+' | '+df_occ_metric['occ_score2'].astype(str)


    df_occ_metric =df_occ_metric.pivot(index="y_axis", columns="x_axis", values="occ_score").fillna(value = '-')
    print("Occlusion Metric")
    df_occ_metric= df_occ_metric.style.set_properties(**{'text-align': 'center'})
    df_occ_metric = df_occ_metric.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    display(df_occ_metric)
    print("\n")
    print("\n")
    ###############################################################################################################

    df_fn_unique1 = df_fn_unique1.sort_values(by =['count'],ascending=False)
    df_fn_unique1 = df_blank.append(df_fn_unique1)
    df_fn_unique1['size']=df_fn_unique1['count']*df_fn_unique1['count']
    df_fn_unique1['Color Significance']='Exclusive FN missed by classifier on y axis'


    df_fn_twin['size']=df_fn_twin['count']*df_fn_twin['count']
    df_fn_twin['Color Significance']='Exclusive FN missed by classifier on x & y axis'

    df_final = df_fn_unique1.append(df_fn_twin)

    fig=px.scatter(df_final, x="x_axis", y="y_axis",
               color_discrete_sequence=px.colors.qualitative.Alphabet,
               color = 'Color Significance',
               size='size',
               opacity=0.6,
               size_max=70,
               color_discrete_map={"Exclusive FN missed by classifier on x & y axis": 'red',"Exclusive FN missed by classifier on y axis":'orange'},
              
              )
    fig.update_traces(marker=dict(
                                      line=dict(width=0.2,
                                                color='DarkSlateGrey')),
                         selector=dict(mode='markers'))
    fig.update_layout(
        width=1000,
        height=600
    )
    fig.update_layout(
        xaxis={'side': 'top'}, 
        yaxis={'side': 'left'}  
    )
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="top",
        y=0,
        xanchor="left",
        x=0.1
    ))
    fig.update_layout(title_text='MARS Occlusion Chart', title_x=0.05)

    fig.show()


In [5]:
import pandas as pd
import plotly.express as px
def main():
    df = pd.read_csv('classification_breast_cancer_dataset.csv')
    total_classifiers = df['classifier'].nunique()
    k=(df.shape[0])/total_classifiers
    Shinethrough(df, k)
    Occlusion(df, k)
if __name__ == "__main__":
    main()

ShineThrough Scores(Count of Exclusive True Positives)


x_axis,DecisionTree,LogisticRegression,RandomForest,SVM
y_axis,,,,
DecisionTree,-,1/90 | 1/90,1/90 | 1/90,1/90 | 3/90
LogisticRegression,0/90 | 1/90,-,0/90 | 1/90,0/90 | 2/90
RandomForest,0/90 | 1/90,0/90 | 1/90,-,0/90 | 4/90
SVM,2/90 | 3/90,2/90 | 2/90,2/90 | 4/90,-






ShineThrough Metric


x_axis,DecisionTree,LogisticRegression,RandomForest,SVM
y_axis,,,,
DecisionTree,-,0.01 | 0.01,0.01 | 0.01,0.01 | 0.03
LogisticRegression,0.0 | 0.01,-,0.0 | 0.01,0.0 | 0.02
RandomForest,0.0 | 0.01,0.0 | 0.01,-,0.0 | 0.04
SVM,0.02 | 0.03,0.02 | 0.02,0.02 | 0.04,-
